In [1]:
import sys
import os
import uproot
import glob
import pandas as pd
import numpy as np
import ROOT
import matplotlib.pyplot as plt
import json
os.system("")

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
import yaml

Welcome to JupyROOT 6.18/00


In [2]:
from Tools.PlotTools import *

TrainConfig='Config.yaml'

with open(r''+TrainConfig+'') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    Conf = yaml.load(file,Loader=yaml.FullLoader)

In [3]:
if Conf['Debug']==True:
    prGreen("Running in debug mode : Only every 10th event will be used")

 Running in debug mode : Only every 10th event will be used


In [4]:
if len(Conf['MVAs'])>0:
    for key in Conf['MVAs']:
        os.system("mkdir -p " + Conf['OutputDirName']+"/"+str(key))
prGreen("Making output directory")
os.system("mkdir -p " + Conf['OutputDirName'])
os.system("mkdir -p " + Conf['OutputDirName']+"/CodeANDConfig")
os.system("mkdir -p " + Conf['OutputDirName']+"/Thresholds")
os.system("cp "+TrainConfig+" ./"+ Conf['OutputDirName']+"/CodeANDConfig/")
os.system("cp Config.yaml ./"+ Conf['OutputDirName']+"/CodeANDConfig/")

 Making output directory


0

In [5]:
prGreen("You have chosen these MVAs")
print(list(Conf['MVAs'].keys()))
prGreen("You have these classes")
print(list(Conf['ClassKinds'].keys()))

 You have chosen these MVAs
['XGB_1', 'XGB_2', 'DNN_1']
 You have these classes
['DYJets', 'bcTOEQCD', 'GJet']


In [6]:
cat='EleType'
weight="NewWt"
labels=list(Conf['ClassKinds'].keys())

In [7]:
print('Using Keras version: ', tf.keras.__version__)

do_model_fit = 1
number_of_classes = len(list(Conf['ClassKinds'].keys()))
# Create instance of output directory where all results are saved.
output_directory = Conf['OutputDirName']
od = output_directory

Using Keras version:  2.2.4-tf


In [8]:
if not Conf['LoadSavedDataFrame']:
    df=pd.DataFrame()
    for i,ClassKind in enumerate(list(Conf['ClassKinds'].keys())):
        FileLocations=list(Conf['ClassKinds'][ClassKind]['FileLocations'])
        Trees=list(Conf['ClassKinds'][ClassKind]['Trees'])
        Cuts=list(Conf['ClassKinds'][ClassKind]['Cuts'])
        Wts=list(Conf['ClassKinds'][ClassKind]['Wts'])
        for Loc,Tree,Cut,Wt in zip(FileLocations,Trees,Cuts,Wts):
            if Conf['Debug']==True:
                dfi=pd.DataFrame()
                print(ClassKind)
                print(Loc)
                print(Tree)
                print(Cut)
                if Conf['Splitting']['CriteriaBased']:
                    #prGreen("CriteriaBased Splitting and Debug is True")
                    dftri=uproot.open(Loc).get(Tree).pandas.df().query(str(Cut)+' & '+str(Conf['CommonCut'])+' & '+str(Conf['Splitting']['Train_selection_criteria'])).iloc[::10]
                    dftri['Dataset']='Train'
                    dftri['TrainDataset']=1
                    dftei=uproot.open(Loc).get(Tree).pandas.df().query(str(Cut)+' & '+str(Conf['CommonCut'])+' & '+str(Conf['Splitting']['Test_selection_criteria'])).iloc[::10]
                    dftei['Dataset']='Test'
                    dftei['TrainDataset']=0
                    dfi = pd.concat([dftri,dftei],ignore_index=True, sort=False)
                else:
                    #prGreen("Size based Splitting and Debug is True")
                    dfi = uproot.open(Loc).get(Tree).pandas.df().query(str(Cut)+' & '+str(Conf['CommonCut'])).iloc[::10]
                    print(len(dfi.index.values.tolist()))
                    from sklearn.model_selection import train_test_split
                    TrainIndices, TestIndices = train_test_split(dfi.index.values.tolist(), test_size=Conf['Splitting']['testsize'], random_state=Conf['RandomState'], shuffle=True)
                    dfi.loc[TrainIndices,'Dataset'] = "Train"
                    dfi.loc[TestIndices,'Dataset'] = "Test"
                    dfi.loc[TrainIndices,'TrainDataset'] = 1
                    dfi.loc[TestIndices,'TrainDataset'] = 0
            else:
                if Conf['Splitting']['CriteriaBased']:
                    #prGreen("CriteriaBased Splitting and Debug is False")
                    dftri=uproot.open(Loc).get(Tree).pandas.df().query(str(Cut)+' & '+str(Conf['CommonCut'])+' & '+str(Conf['Splitting']['Train_selection_criteria']))
                    dftri['Dataset']='Train'
                    dftri['TrainDataset']=1
                    dftei=uproot.open(Loc).get(Tree).pandas.df().query(str(Cut)+' & '+str(Conf['CommonCut'])+' & '+str(Conf['Splitting']['Test_selection_criteria']))
                    dftei['Dataset']='Test'
                    dftei['TrainDataset']=0
                    dfi = pd.concat([dftri,dftei],ignore_index=True, sort=False)                
                else:
                    #prGreen("Size based Splitting and Debug is False")
                    dfi = uproot.open(Loc).get(Tree).pandas.df().query(str(Cut)+' & '+str(Conf['CommonCut']))
                    print(len(dfi.index.values.tolist()))
                    from sklearn.model_selection import train_test_split
                    TrainIndices, TestIndices = train_test_split(dfi.index.values.tolist(), test_size=Conf['Splitting']['testsize'], random_state=Conf['RandomState'], shuffle=True)
                    dfi.loc[TrainIndices,'Dataset'] = "Train"
                    dfi.loc[TestIndices,'Dataset'] = "Test"
                    dfi.loc[TrainIndices,'TrainDataset'] = 1
                    dfi.loc[TestIndices,'TrainDataset'] = 0
            dfi['xsecwt']=Wt
            dfi[cat]=i
            df=pd.concat([df,dfi],ignore_index=True, sort=False)
            if Conf['SaveDataFrame']:
                df.to_csv('./'+ Conf['OutputDirName'] + '/dfout.zip', index=False,compression='infer')

else:
    df = pd.read_csv('./'+ Conf['OutputDirName'] + '/dfout.zip')

/cvmfs/sft.cern.ch/lcg/views/LCG_96python3/x86_64-centos7-gcc8-opt/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (105) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
TrainIndices=df.index[df['TrainDataset'] == 1].tolist()
prGreen("Size of training dataset = "+str(len(TrainIndices))+" events/objects")
TestIndices=df.index[df['TrainDataset'] == 0].tolist()
prGreen("Size of testing dataset = "+str(len(TestIndices))+" events/objects")

 Size of training dataset = 85982 events/objects
 Size of testing dataset = 21498 events/objects


In [11]:
for MVA in list(Conf['MVAs'].keys()):
    if 'XGB' in MVA:

XGB_1
XGB_2
